In [1]:
import requests

In [5]:
# function to get data from FTX exchange API
api_prefix = "https://ftx.com/api"

def send_get_request(endpoint):
    api_prefix = "https://ftx.com/api"
    url = api_prefix + endpoint
    response = requests.request("GET", url)
    return response.json()

In [7]:
# get data for all markets
all_markets = send_get_request("/markets")

In [42]:
# get historical prices for single market
# GET /markets/{market_name}/candles?resolution={resolution}&start_time={start_time}&end_time={end_time}
# resolution = window length in seconds. start_time and end_time optional timestamps
historical_prices = send_get_request("/markets/BTC-PERP/candles?resolution=3600")

In [43]:
# calculate simple moving averages - input data as slice FTX result and "open", "high", "low", "close"
def calc_sma(data, ohlc):
    periods = len(data)
    sum_price = 0
    for i in range(periods):
        sum_price += data[i][ohlc]
    avg = sum_price / periods
    return avg
    
sma_20 = calc_sma(historical_prices["result"][-20:], "close")
print(sma_20)

40011.0


In [44]:
# calculate exponential moving averages - input data as FTX result, no. of periods and "open", "high", "low", "close"
def calc_next_ema(today, yesterday, multiplier):
    ema = (today * multiplier) + (yesterday * (1-multiplier))
    return ema

def calc_ema(data, periods, ohlc):
    ema = calc_sma(data[0:periods], ohlc)
    multiplier = 2/(periods+1)
    for i in range(periods, len(data)):
        ema = calc_next_ema(data[i][ohlc], ema, multiplier)
    return ema
    
ema_20 = calc_ema(historical_prices["result"], 20, "close")
print(ema_20)

39851.78006512375
